In [1]:
import os
import time

from functions.input2dataframe import *

from functions.analyse_df_pcap import *
from functions.tcpdump_filter_anon import *

from functions.analyse_df_nfdump import *
from functions.nfdump_filter_anon import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
def ddos_analysis_fingerprint_filter_sanitization(input_file, debug=True):
    
    start = time.time()

    if debug: print 'Converting input file to dataframe (time:',  
    df = input2dataframe(input_file) #input2dataframe checks the input_file type (pcap, pcapng, nfdump) and calls the right converter (e.g. pcap2dataframe, nfdump2dataframe)
    if debug: print str(time.time() - start)+')'
    
    #########################################################        
    #########################################################
    start = time.time()
    if debug: print 'Analysing the dataframe for finding attack patterns...'    
        
    if 'tuple' in str(type(df)):
        all_patterns = analyse_df_tcpdump(df, debug)
        if debug: print 'Dataframe completely analysed and all attack patterns identified! (time:'+str(time.time() - start)+')'
        #########################################################        
        #########################################################
        start = time.time()
        if debug: print 'Generating attack vectors (with the same format as the input file) and anonymizing the victim information (dst_ip) (time:' 
        for pattern in all_patterns["patterns"]:
            nfdump_filter_anon(input_file,  pattern, all_patterns['dst_ip'])
        #########################################################        
        #########################################################   
    else:
        all_patterns = analyse_df_pcap(df, debug=True)
        display(all_patterns)
        
        if debug: print 'Dataframe completely analysed and all attack patterns identified! (time:'+str(time.time() - start)+')'
        #########################################################        
        #########################################################
        start = time.time()
        if debug: print 'Generating attack vectors (same format as input file) and anonymizing victim\'s information (dst_ip) (time:' 
        for pattern in all_patterns["patterns"]:
            tcpdump_filter_anon(input_file, pattern, all_patterns["dst_ip"])
            
    if debug: print str(time.time() - start)+')'
#     #########################################################        
#     #########################################################
#     #TODO: Generate rule/signatures for specific detection and mitigation technologies (SIDS: BRO, SNORT, SURICATA; Firewall)
    

## Execution from Jupyter Notebook

In [3]:
input_file="input4test/1.pcap"
ddos_analysis_fingerprint_filter_sanitization(input_file)

Converting input file to dataframe (time: 0.127379894257)
Analysing the dataframe for finding attack patterns...
Total number packets: 3739

###################################
IDENTIFYING MAIN CHARACTERISTICS:
###################################
Target (destination) IP: 172.31.28.132
Number of packets: 2559

###################################################################################################################
IP protocol used in packets going to target IP: 17
Number of packets: 2547

#############################
PORT FREQUENCY OF REMAINING PACKETS
##############################
SOURCE ports frequency
53       46.014920
32817    23.557126
1900     15.508441
2051      5.261091
3326      4.004711
Name: src_port, dtype: float64

DESTINATION ports frequency
80       53.985080
53680     0.235571
49105     0.235571
50027     0.157048
48615     0.157048
Name: dst_port, dtype: float64

 Using top dest port:  80

############################
PATTERN (ATTACK VECTOR) LABEL 
########

{'dst_ip': '172.31.28.132',
 'patterns': [{'dst_ports': {80: 100.0},
   'end_timestamp': 1419864098.046775,
   'ip_protocol': 17,
   'pattern_packet_count': 1375,
   'pattern_traffic_share': 53.731926533802266,
   'selected_port': 'dst80',
   'src_ips': ['200.45.216.96',
    '171.37.157.18',
    '122.137.117.247',
    '180.123.27.16',
    '124.236.1.161',
    '184.43.33.18',
    '184.39.242.182',
    '124.125.46.8',
    '172.3.83.62',
    '162.198.246.54',
    '162.233.186.54',
    '172.6.53.230',
    '172.12.42.206',
    '172.5.178.166',
    '162.233.186.158',
    '172.6.53.94',
    '172.13.243.182',
    '172.6.52.118',
    '172.13.23.118',
    '172.3.83.198',
    '172.14.81.70',
    '172.10.106.230',
    '162.224.246.30',
    '172.10.98.222',
    '162.196.242.62',
    '172.10.3.246',
    '162.203.12.182'],
   'src_ports': {1900: 28.727272727272727,
    2051: 9.745454545454546,
    3326: 7.418181818181818,
    3329: 0.14545454545454545,
    3332: 2.1818181818181817,
    7200: 1.018181

Dataframe completely analysed and all attack patterns identified! (time:0.131577968597)
Generating attack vectors (same format as input file) and anonymizing victim's information (dst_ip) (time:
20141229_dst80.pcap  20141229_src53.pcap  temp
0.161283969879)


## Main for python script 

In [ ]:
if __name__ == '__main__':
    import argparse
    import os.path
    
    parser = argparse.ArgumentParser(description='')
    
    parser.add_argument('--input', metavar='input_file', required=True,
                        help='Path of a input file')
    args = parser.parse_args()
    
    input_file=args.input
    
    if os.path.isfile(input_file) == False:
        print 'We were unable to find the file. Please check the file path!!'
    
        ddos_analysis_fingerprint_filter_sanitization(input_file, debug=True)